## Цель работы

Найти точку минимума функции, заданной в соответствии с вариантом, методом золотого сечения и методом квадратичной интерполяции.

## Исходные данные

Дана функция: $$f(x) = \frac{1}{7}x^7 - x^3 + \frac{1}{2}x^2 - x$$

Рассмотреть отрезок $[a, b] = [1, 1.5]$. Необходимая точность вычислений для итеративного метода золотого сечения $\varepsilon = 0.05$.

In [ ]:
f = lambda x: (1./7.)*x**7 - x**3 + (1./2.)*x**2 - x
int_a, int_b = [1, 1.5]
eps = 0.05

## График функции

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

xs = np.arange(int_a - 0.05, int_b + 0.05, step=0.001)
plt.figure(figsize=(7, 4))
plt.plot(xs, [f(x) for x in xs])
plt.axvline(x=int_a, color='k', linestyle='--')
plt.axvline(x=int_b, color='k', linestyle='--')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid()
plt.show()

## Метод золотого сечения

In [ ]:
%load_ext cython

In [ ]:
%%cython
from libc.math cimport abs, sqrt

def gsect_search(f, double a, double b, double delta):
    cdef double phi_inv = (sqrt(5.) - 1.) / 2. # 1 / phi, inverse of golden ratio
    cdef double a1, b1
    cdef int iter_count = 0
    while True:
        iter_count += 1

        a1 = a + (b - a)*(1 - phi_inv)
        b1 = a + (b - a)*phi_inv

        if f(a1) < f(b1):
            b = b1
        else:
            a = a1

        print(f'Iter {iter_count}: a={a}, b={b}')

        if abs(a - b) <= delta:
            xmin = (a + b) / 2.
            return xmin, f(xmin), iter_count

In [ ]:
xmin, ymin, iter_count = gsect_search(f, int_a, int_b, eps)
print(f'Результат вычисления методом золотого сечения: xmin={xmin:.6f}, ymin={ymin:.6f}')
print(f'Число итераций: {iter_count}')

### Ручная проверка

Вычислим состояние $a$, $b$ на пяти шагах:
1. $a = 1.191, b = 1.5$
2. $a = 1.191, b = 1.382$
3. $a = 1.264, b = 1.382$
4. $a = 1.264, b = 1.337$
5. $a = 1.264, b = 1.31$


## Квадратичная интерполяция

Необходимо взять три точки $(x_1, y_1), (x_2, y_2), (x_3, y_3)$, удовлетворяющие следующим условиям:
$$x_1 < x_2 < x_3,\ f(x_1) \geqslant f(x_2) \leqslant f(x_3)$$

В качестве $x_1$ и $x_3$ положим границы отрезка $[a, b]$, а $x_2$ выберем по графику: $x_2 = 1.2$.

In [ ]:
%%cython
def lagrange(f, double x1, double x2, double x3):
    cdef double y1 = f(x1), y2 = f(x2), y3 = f(x3)
    cdef double delta = (x2 - x1)*(x3 - x1)*(x3 - x2)
    cdef double A = ((x3 - x2)*y1 - (x3 - x1)*y2 + (x2 - x1)*y3) / delta
    cdef double B = (-(x3**2 - x2**2)*y1 + (x3**2 - x1**2)*y2 - (x2**2 - x1**2)*y3) / delta
    cdef double C = (x2*x3*(x3 - x2)*y1 - x1*x3*(x3 - x1)*y2 + x1*x2*(x2 - x1)*y3) / delta
    cdef double xmin = -B / (2*A)
    cdef double ymin = -B**2 / (4*A) + C
    return xmin, ymin

In [ ]:
xmin, ymin = lagrange(f, x1=int_a, x2=1.3, x3=int_b)
print(f'Результат вычисления методом квадратичной интерполяции: xmin={xmin:.6f}, ymin={ymin:.6f}')